# NY CitiBike Analysis: Sourcing Data with an API

## Import Libraries and Load CitiBike Data

In [1]:
# Import libraries
import pandas as pd 
import numpy as np
import os 
import requests
import json
from datetime import datetime

In [2]:
# Create a list with all files in the folder using a list comprehension
folderpath = r'/Users/emilycurcio/NY_CitiBike_Analysis/Raw Data'
filepaths  = [os.path.join(folderpath, name) for name in os.listdir(folderpath)]
filepaths

['/Users/emilycurcio/NY_CitiBike_Analysis/Raw Data/202208-citibike-tripdata_3.csv',
 '/Users/emilycurcio/NY_CitiBike_Analysis/Raw Data/202208-citibike-tripdata_2.csv',
 '/Users/emilycurcio/NY_CitiBike_Analysis/Raw Data/202210-citibike-tripdata_4.csv',
 '/Users/emilycurcio/NY_CitiBike_Analysis/Raw Data/202208-citibike-tripdata_1.csv',
 '/Users/emilycurcio/NY_CitiBike_Analysis/Raw Data/202210-citibike-tripdata_1.csv',
 '/Users/emilycurcio/NY_CitiBike_Analysis/Raw Data/.DS_Store',
 '/Users/emilycurcio/NY_CitiBike_Analysis/Raw Data/202208-citibike-tripdata_4.csv',
 '/Users/emilycurcio/NY_CitiBike_Analysis/Raw Data/202203-citibike-tripdata_2.csv',
 '/Users/emilycurcio/NY_CitiBike_Analysis/Raw Data/202210-citibike-tripdata_2.csv',
 '/Users/emilycurcio/NY_CitiBike_Analysis/Raw Data/202203-citibike-tripdata_1.csv',
 '/Users/emilycurcio/NY_CitiBike_Analysis/Raw Data/202210-citibike-tripdata_3.csv',
 '/Users/emilycurcio/NY_CitiBike_Analysis/Raw Data/202204-citibike-tripdata_1.csv',
 '/Users/emil

In [3]:
# Remove the .DS_Store file from the list
del filepaths[5]
filepaths

['/Users/emilycurcio/NY_CitiBike_Analysis/Raw Data/202208-citibike-tripdata_3.csv',
 '/Users/emilycurcio/NY_CitiBike_Analysis/Raw Data/202208-citibike-tripdata_2.csv',
 '/Users/emilycurcio/NY_CitiBike_Analysis/Raw Data/202210-citibike-tripdata_4.csv',
 '/Users/emilycurcio/NY_CitiBike_Analysis/Raw Data/202208-citibike-tripdata_1.csv',
 '/Users/emilycurcio/NY_CitiBike_Analysis/Raw Data/202210-citibike-tripdata_1.csv',
 '/Users/emilycurcio/NY_CitiBike_Analysis/Raw Data/202208-citibike-tripdata_4.csv',
 '/Users/emilycurcio/NY_CitiBike_Analysis/Raw Data/202203-citibike-tripdata_2.csv',
 '/Users/emilycurcio/NY_CitiBike_Analysis/Raw Data/202210-citibike-tripdata_2.csv',
 '/Users/emilycurcio/NY_CitiBike_Analysis/Raw Data/202203-citibike-tripdata_1.csv',
 '/Users/emilycurcio/NY_CitiBike_Analysis/Raw Data/202210-citibike-tripdata_3.csv',
 '/Users/emilycurcio/NY_CitiBike_Analysis/Raw Data/202204-citibike-tripdata_1.csv',
 '/Users/emilycurcio/NY_CitiBike_Analysis/Raw Data/202204-citibike-tripdata_

In [4]:
# Read and concatenate all files simultaneously
df = pd.concat((pd.read_csv(f) for f in filepaths), ignore_index = True)

/var/folders/4n/51fny6wj15lbcvykgf0dns700000gn/T/ipykernel_84104/4167960515.py:2: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(f) for f in filepaths), ignore_index = True)
/var/folders/4n/51fny6wj15lbcvykgf0dns700000gn/T/ipykernel_84104/4167960515.py:2: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(f) for f in filepaths), ignore_index = True)
/var/folders/4n/51fny6wj15lbcvykgf0dns700000gn/T/ipykernel_84104/4167960515.py:2: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(f) for f in filepaths), ignore_index = True)
/var/folders/4n/51fny6wj15lbcvykgf0dns700000gn/T/ipykernel_84104/4167960515.py:2: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(f) for f 

In [5]:
df.shape

(30689921, 13)

#### In order to load the data from multiple csv files I start by making sure those csv files are all in the same folder on my computer, and then I identify that folder path and assign it to the variable folderpath. Next I use the join() function and a for loop to grab all the names of the csv files and append them each to the folderpath, storing this list in the variable filepaths. Lastly, I use a for loop again to read each of the csv files in the filepaths list, and then use the concat() function to merge the data vertically.

#### When merging the data, I choose to use a generator (parenthesis) instead of a list comprehension (brackets). This is because I only need to iterate through the files once, and generators are capable of handling one file at a time, making them more memory efficient.

## Get Weather Data using NOAA's API

In [6]:
# Define your NOAA token
Token = os.getenv("NOAA_TOKEN")

In [7]:
# Get the API 
r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USW00014732&startdate=2022-01-01&enddate=2022-12-31', headers = {'token': Token})

In [8]:
# Load the API response as a json
d = json.loads(r.text)

In [9]:
# Secure all items in the response that correspond to TAVG
avg_temps = [item for item in d['results'] if item['datatype'] == 'TAVG']

In [10]:
# Get only the date field from all average temperature readings
dates_temp = [item['date'] for item in avg_temps]

In [11]:
# Get the temperature from all average temperature readings
temps = [item['value'] for item in avg_temps]

In [12]:
# Create a dataframe for the results
df_temp = pd.DataFrame()

In [13]:
# Get only date and cast it to date time
# Convert temperature from tenths of Celsius to normal Celsius
df_temp['date'] = [datetime.strptime(d, '%Y-%m-%dT%H:%M:%S') for d in dates_temp]
df_temp['avgTemp'] = [float(v)/10.0 for v in temps]

In [14]:
df_temp.head()

,date,avgTemp
0,2022-01-01,11.6
1,2022-01-02,11.4
2,2022-01-03,1.4
3,2022-01-04,-2.7
4,2022-01-05,3.2


In [15]:
# Export weather data to a csv file
df_temp.to_csv('weather_data.csv')

## Merge CitiBike and Weather Dataframes

In [16]:
df.head(1)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,DF208007EE5F9D79,classic_bike,2022-08-26 15:21:44,2022-08-26 15:34:20,Berkeley Pl & 6 Ave,4134.06,Windsor Pl & 8 Ave,3620.02,40.67653,-73.978469,40.660906,-73.983074,member


In [17]:
# Format started_at column in CitiBike data to match date column in weather data
df['started_at'] = pd.to_datetime(df['started_at'], dayfirst = True)

/var/folders/4n/51fny6wj15lbcvykgf0dns700000gn/T/ipykernel_84104/480626359.py:2: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['started_at'] = pd.to_datetime(df['started_at'], dayfirst = True)


In [18]:
df['date'] = pd.to_datetime(df['started_at'], format='%Y-%m-%d').dt.date

In [19]:
df['date'] = pd.to_datetime(df['date'])

In [26]:
%%time
df_merged = df.merge(df_temp, how = 'left', on = 'date', indicator = True)

CPU times: user 48 s, sys: 3min 16s, total: 4min 4s
Wall time: 10min 34s


In [27]:
df_merged.shape

(30689921, 16)

In [28]:
df_merged.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,date,avgTemp,_merge
0,DF208007EE5F9D79,classic_bike,2022-08-26 15:21:44,2022-08-26 15:34:20,Berkeley Pl & 6 Ave,4134.06,Windsor Pl & 8 Ave,3620.02,40.676530,-73.978469,40.660906,-73.983074,member,2022-08-26,27.4,both
1,48A796B4996BACB0,electric_bike,2022-08-03 12:56:29,2022-08-03 13:06:10,21 St & 4 Ave,3668.04,Degraw St & Smith St,4298.05,40.662584,-73.995554,40.682915,-73.993182,member,2022-08-03,27.6,both
2,AE459F435C0B1D5B,classic_bike,2022-08-11 17:49:47,2022-08-11 18:05:08,48 St & Barnett Ave,6358.05,Queens Plaza North & Crescent St,6429.01,40.750016,-73.915409,40.751102,-73.940737,member,2022-08-11,26.6,both
3,CA8B4A94620E74C0,electric_bike,2022-08-10 17:25:41,2022-08-10 17:35:23,7 Ave & Central Park South,6912.01,8 Ave & W 33 St,6450.12,40.766741,-73.979069,40.751551,-73.993934,member,2022-08-10,27.2,both
4,EB85C6F0148BC348,classic_bike,2022-08-31 13:47:12,2022-08-31 14:24:00,Jerome Ave & W 195 St,8653.09,Sherman Ave & E 166 St,8072.06,40.868858,-73.896500,40.831305,-73.917448,member,2022-08-31,25.6,both


In [29]:
# Check for 
df_merged['_merge'].value_counts(dropna = False)

_merge
both          30689921
left_only            0
right_only           0
Name: count, dtype: int64

In [30]:
# Remove _merge column
df_merged.drop('_merge', axis = 1, inplace = True)

In [31]:
df_merged.to_csv('new_york_data.csv')